In [2]:
#테트리스
# """ tetris  """
import sys
from math import sqrt    #제곱근(√) 계산을 쓰기 위해 math 모듈에서 불러오는 코드예
from random import randint
import pygame
from pygame.locals import QUIT, KEYDOWN,K_LEFT, K_RIGHT, K_SPACE,K_DOWN

BLOCK_DATA = (
    (
        (0, 0, 1, \
         1, 1, 1, \
         0, 0, 0),
        (0, 1, 0, \
         0, 1, 0, \
         0, 1, 1),
        (0, 0, 0, \
         1, 1, 1, \
         1, 0, 0),
        (1, 1, 0, \
         0, 1, 0, \
         0, 1, 0),
    ), (
        (2, 0, 0, \
         2, 2, 2, \
         0, 0, 0),
        (0, 2, 2, \
         0, 2, 0, \
         0, 2, 0),
        (0, 0, 0, \
         2, 2, 2, \
         0, 0, 2),
        (0, 2, 0, \
         0, 2, 0, \
         2, 2, 0)
    ), (
        (0, 3, 0, \
         3, 3, 3, \
         0, 0, 0),
        (0, 3, 0, \
         0, 3, 3, \
         0, 3, 0),
        (0, 0, 0, \
         3, 3, 3, \
         0, 3, 0),
        (0, 3, 0, \
         3, 3, 0, \
         0, 3, 0)
    ), (
        (4, 4, 0, \
         0, 4, 4, \
         0, 0, 0),
        (0, 0, 4, \
         0, 4, 4, \
         0, 4, 0),
        (0, 0, 0, \
         4, 4, 0, \
         0, 4, 4),
        (0, 4, 0, \
         4, 4, 0, \
         4, 0, 0)
    ), (
        (0, 5, 5, \
         5, 5, 0, \
         0, 0, 0),
        (0, 5, 0, \
         0, 5, 5, \
         0, 0, 5),
        (0, 0, 0, \
         0, 5, 5, \
         5, 5, 0),
        (5, 0, 0, \
         5, 5, 0, \
         0, 5, 0)
    ), (
        (6, 6, 6, 6),
        (6, 6, 6, 6),
        (6, 6, 6, 6),
        (6, 6, 6, 6)
    ), (
        (0, 7, 0, 0, \
         0, 7, 0, 0, \
         0, 7, 0, 0, \
         0, 7, 0, 0),
        (0, 0, 0, 0, \
         7, 7, 7, 7, \
         0, 0, 0, 0, \
         0, 0, 0, 0),
        (0, 0, 7, 0, \
         0, 0, 7, 0, \
         0, 0, 7, 0, \
         0, 0, 7, 0),
        (0, 0, 0, 0, \
         0, 0, 0, 0, \
         7, 7, 7, 7, \
         0, 0, 0, 0)
    )
)


class Block:
    #블록객체
    def __init__(self,count):
        self.turn = randint(0,3)  #블록의 방향 좌,우,아래
        self.type = BLOCK_DATA[randint(0,6)] #블록 6가지 의 2차원데이터(4방향분)
        self.data = self.type[self.turn]    #블록 1차원데이터 (현재의 방향만)
        self.size = int(sqrt(len(self.data)))#블록의크기
                                            #sqrt()는 “이 블록이 몇 칸짜리 정사각형인지” 알려주는 계산이에요.
                                            #sqrt()는 1차원 데이터를 2차원으로 펼칠 때의 한 변 길이를 계산하는 도구예요.
        self.xpos = randint(2,8-self.size)  # 블록의 x좌표
        self.ypos = 1 - self.size           #블록의 y 좌표        
        self.fire = count + INTERVAL        #낙하시작시간

    def update(self,count):  #블록의 낙하를 처리한다
        erased= 0
        if is_overlapped(self.xpos,self.ypos +1,self.turn): #블록이 겹치는지 아닌지 여부를 반화하는 함수이다
                            # ypos에 +1을 더해서 겹침을 확인하고 겹침상태이면 더이상 낙하할 수 없으므로 BLOCK데이터를,즉 자신을 FIELD에 복사한다.
            for y_offset in range(BLOCK.size):
                for x_offset in range(BLOCK.size):
                    if 0 <= self.xpos + x_offset <WIDTH and 0 <= self.ypos +y_offset <HEIGHT:
                        val= BLOCK.data[y_offset*BLOCK.size + x_offset]

                        if val != 0:
                            FIELD[self.ypos+y_offset][self.xpos+x_offset] = val
            erased = erase_line()
            go_next_block(count)
        if self.fire < count:
            self.fire = count + INTERVAL
            self.ypos += 1
        return erased

    def draw(self):  #블록을 그린다.
        for index in range(len(self.data))   :
            xpos = index % self.size
            ypos = index  // self.size
            val= self.data[index]
            if 0 <= ypos + self.ypos < HEIGHT and 0<= xpos + self.xpos <WIDTH and val !=0:
                x_pos= 25 + (xpos + self.xpos) *25
                y_pos = 25 + (ypos + self.ypos) *25
                pygame.draw.rect(SURFACE,COLORS[val],(x_pos,y_pos,24,24))
def erase_line():
    #행이 모두찬 단을 지운다
    erased = 0
    ypos = 20
    while ypos >= 0:
        if all(FIELD[ypos]):
            erased += 1
            del FIELD[ypos]
            FIELD.insert(0,[8,0,0,0,0,0,0,0,0,0,0,8])
        else:
            ypos -=1
    return erased

def is_game_over():
    filled = 0
    for cell in FIELD[0]:
        if cell !=0 :
            filled += 1
    return filled > 2 #2 = 좌우의 벽

def go_next_block(count):
    # 다음블록으로 전환한다
    global BLOCK,NEXT_BLOCK
    BLOCK = NEXT_BLOCK if NEXT_BLOCK != None else Block(count)
    NEXT_BLOCK =Block(count)

def is_overlapped(xpos,ypos,turn):
    #블록이 벽이나 땅의 블록과 충돌하는지 아닌지
    data = BLOCK.type[turn]
    for y_offset in range(BLOCK.size):
        for x_offset in range(BLOCK.size):
            if 0 <= xpos+x_offset <WIDTH and  0 <= ypos + y_offset <HEIGHT:
                if data[y_offset*BLOCK.size + x_offset] !=0 and FIELD[ypos+y_offset][xpos+x_offset] !=0:
                    return True
    return False


pygame.init()
pygame.key.set_repeat(30, 30)
SURFACE = pygame.display.set_mode((600, 600))
FPSCLOCK = pygame.time.Clock()
WIDTH = 12
HEIGHT = 22
INTERVAL= 40    #블록의 일정한 낙하간격
FIELD = [[0 for _ in range(WIDTH)] for _ in range(HEIGHT)] #벽과 겹쳐진 블록의 상태를 유지하는 2차원데이터
COLORS = ((0,0,0),(255,165,0),(0,0,255),(0,255,255),(0,255,0),(255,0,255),(255,255,0),(255,0,0),(128,128,128))  #8색
BLOCK = None    #현재 낙하중인 블록
NEXT_BLOCK = None  # 다음에 낙하하는 블록

def main():
    """ 메인 루틴 """
    global INTERVAL
    count = 0
    score = 0
    game_over = False

    smallfont = pygame.font.SysFont(None, 36)
    largefont = pygame.font.SysFont(None, 72)
    
    message_over = largefont.render("GAME OVER!!",True, (0, 255, 225))
    message_rect = message_over.get_rect()
    message_rect.center = (300, 300)

    go_next_block(INTERVAL)
    
    for ypos in range(HEIGHT):
        for xpos in range(WIDTH):
            FIELD[ypos][xpos] = 8 if xpos ==0 or xpos ==WIDTH-1 else 0
    for index in range(WIDTH):
        FIELD[HEIGHT-1][index] = 8
    

    while True:
        key = None
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                key = event.key
        game_over = is_game_over()
        if not game_over:
            count +=5
            if count % 1000 == 0:
                INTERVAL = max(1,INTERVAL-2)
            erased= BLOCK.update(count)

            if erased > 0:
                score += (2**erased) * 100
            #키이벤트 처리
            next_x,next_y,next_t = BLOCK.xpos,BLOCK.ypos,BLOCK.turn
            if key == K_SPACE:
                next_t =(next_t +1) %4
            elif key ==K_RIGHT:
                next_x += 1
            elif key ==K_LEFT:
                next_x -= 1
            elif key == K_DOWN:
                next_y += 1

            if not is_overlapped(next_x,next_y,next_t):
                BLOCK.xpos = next_x
                BLOCK.ypos = next_y
                BLOCK.turn = next_t
                BLOCK.data = BLOCK.type[BLOCK.turn]
        
        # 전체 낙하중인 블록그리기
        SURFACE.fill((0, 0, 0))
        for ypos in range(HEIGHT):
            for xpos in range(WIDTH):
                val = FIELD[ypos][xpos]
                pygame.draw.rect(SURFACE,COLORS[val],(xpos*25+25,ypos*25+25,24,24))
        BLOCK.draw()
        #다음 블록 그리기
        for ypos in range(NEXT_BLOCK.size):
            for xpos in range(NEXT_BLOCK.size):
                val = NEXT_BLOCK.data[xpos + ypos*NEXT_BLOCK.size]
                pygame.draw.rect(SURFACE,COLORS[val],(xpos*25+460,ypos*25+100,24,24))
        #점수나타내기
        score_str = str(score).zfill(6)
        score_image = smallfont.render(score_str,True, (0, 255, 0))
        SURFACE.blit(score_image, (500, 30))

        if game_over:
            SURFACE.blit(message_over, message_rect)

        pygame.display.update()
        FPSCLOCK.tick(15)

if __name__ == '__main__':
    main()


SystemExit: 